In [1]:
import numpy as np
import cupy as cp
print("CuPy version:", cp.__version__)
print("CUDA runtime version:", cp.cuda.runtime.runtimeGetVersion())
import tifffile as tiff
from scipy.ndimage import distance_transform_edt
from skan import Skeleton
from joblib import Parallel, delayed
import sys

# Load binary and skeleton volumes
binary_volume = tiff.imread("outputs/output_volume.tif")
skeleton_volume = tiff.imread("outputs/output_skeleton_volume.tif")

print(f'Size of binary volume: {sys.getsizeof(binary_volume)} bytes')
print(f'Size of skeleton volume: {sys.getsizeof(skeleton_volume)} bytes')

# Transfer binary and skeleton volumes to GPU
binary_volume_gpu = cp.asarray(binary_volume)
skeleton_volume_gpu = cp.asarray(skeleton_volume)
print("Volumes transfered to gpu")

def compute_statistics(skeleton):
    """
    Computes mean, max, and median widths for each path in parallel.
    """
    def path_stats(path_id):
        # Retrieve thickness (width) values for this path
        widths = skeleton.path_with_data(path_id)[1]
        return np.mean(widths), np.max(widths), np.median(widths)
    
    # Use parallel processing to calculate statistics for each path
    results = Parallel(n_jobs=-1)(delayed(path_stats)(i) for i in range(skeleton.n_paths))
    means, maxs, medians = zip(*results)
    return means, maxs, medians

def calculate_local_thickness_gpu(binary_volume_gpu, skeleton_volume_gpu, max_distance=10):
    """
    Calculate a masked distance map near the skeleton using GPU.
    """
    # Calculate distance transform of the binary volume on GPU
    masked_distance_map_gpu = cp.ndimage.distance_transform_edt(binary_volume_gpu)
    
    # Mask distances above max_distance
    masked_distance_map_gpu[masked_distance_map_gpu > max_distance] = 0
    print(f'Size of masked distance map (on GPU): {masked_distance_map_gpu.nbytes} bytes')
    
    # Apply the skeleton mask
    return masked_distance_map_gpu * skeleton_volume_gpu

# Calculate the masked local thickness map on the GPU
masked_thickness_map_gpu = calculate_local_thickness_gpu(binary_volume_gpu, skeleton_volume_gpu)
print("Masked local thickness calculated on GPU")

# Convert back to CPU (NumPy array) if further processing is needed on CPU
masked_thickness_map = cp.asnumpy(masked_thickness_map_gpu)
print("Converted to CPU...")

# Analyze skeleton data using skan (assuming skan needs CPU-based processing)
skeleton = Skeleton(cp.asnumpy(skeleton_volume_gpu))
means, maxs, medians = compute_statistics(skeleton)

print("Statistics computed for each path in the skeleton.")
print("Mean thicknesses:", means)
print("Max thicknesses:", maxs)
print("Median thicknesses:", medians)


CuPy version: 13.3.0
CUDA runtime version: 12060
Size of binary volume: 2666222244 bytes
Size of skeleton volume: 2666222244 bytes
Volumes transfered to gpu


AttributeError: module 'cupy' has no attribute 'ndimage'